# Оценка эффективности каналов привлечения пользователей

____



### Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

 Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.
Примечания:
- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

Описание данных 

- date – дата совершения события

- event - событие:

  - app_install – установка приложения
  - app_start – открыть приложения
  - registration – зарегистрироваться 
  - search – перейти на страницу поиска товаров (каталог)
  - open_item – открыть товар
  - choose_item – отправить товар в корзину
  - tap_basket – перейти в корзину
  - purchase – подтверждение покупки

- gender – пол пользователя

- os_name – платформа пользователя

- city – город пользователя

- device_id – идентификатор устройства пользователя

- urm_source – канал, с которого пришел пользователь:

  - yandex-direct – Яндекс директ
  - google_ads – реклама в Google
  - vk_ads – реклама в ВК
  - instagram_ads – реклама в instagram
  - facebook_ads – реклама в facebook
  - referral – акция «приведи друга»
  - Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  

- purchase_sum – стоимость покупки (при совершении события ‘purchase’)

In [166]:
import pandas as pd 


In [167]:
data = pd.read_csv('C:/Users/рома/Downloads/KC_case_data .csv',
                  parse_dates=['date'])
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [168]:
# some info about data
data.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2747968 entries, 0 to 2747967
Data columns (total 8 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   event         object        
 2   purchase_sum  float64       
 3   os_name       object        
 4   device_id     int64         
 5   gender        object        
 6   city          object        
 7   utm_source    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 167.7+ MB


In [169]:
# missing values 
data.isna().sum()

date                  0
event                 0
purchase_sum    2606585
os_name               0
device_id             0
gender                0
city                  0
utm_source            0
dtype: int64

In [170]:
data.describe(include='all')

<ipython-input-170-589a52936347>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe(include='all')


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
count,2747968,2747968,141383.000000,2747968,2.747968e+06,2747968,2747968,2747968
unique,91,7,NaN,2,NaN,2,2,7
top,2020-01-10 00:00:00,app_start,NaN,android,NaN,female,Moscow,-
freq,89831,748705,NaN,1628119,NaN,1543644,1491481,610458
first,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2020-03-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,709.116092,NaN,1.090841e+07,NaN,NaN,NaN
std,NaN,NaN,1289.458186,NaN,9.871445e+06,NaN,NaN,NaN
min,NaN,NaN,61.000000,NaN,4.013000e+03,NaN,NaN,NaN
25%,NaN,NaN,226.500000,NaN,1.975336e+06,NaN,NaN,NaN


____

## Задача 1
Посчитать MAU за февраль

In [171]:
MAU_february = data.loc[(data.date > '2020-01-31')&(data.date < '2020-03-01')]\
                   .device_id\
                   .nunique()

print(f'Число уникальных активных пользователей в феврале - {MAU_february}')

Число уникальных активных пользователей в феврале - 75032


____

## Задача 2

Посчитать количество установок в январе:

In [172]:
# Посмотрим колонку event чтобы понять какие дейсвия пользователей возможны 
data.event.unique()

array(['app_start', 'choose_item', 'purchase', 'search', 'tap_basket',
       'app_install', 'register'], dtype=object)

In [173]:
# отберем только те записи которые соответсвуют установкам в январе
january_app_installs = data.query('event=="app_install"')\
                           .loc[(data.date >= '2020-01-01')&(data.date < '2020-02-01')]\
                           .device_id\
                           .nunique()

print(f'Количество установок в январе - {january_app_installs}')

Количество установок в январе - 80297


____

## Задача 3

Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей?



In [174]:
# вычисляем дату установки для каждого девайса
install_date = data.query('event=="app_install"')\
                   .groupby('device_id',as_index=False)\
                   .date.min()\
                   .rename(columns={'date':'install_date'})
install_date

,device_id,install_date
0,4013,2020-01-15
1,4016,2020-01-04
2,4046,2020-01-04
3,4065,2020-03-19
4,4067,2020-01-31
...,...,...
154592,35379281,2020-03-29
154593,35380796,2020-03-31
154594,35381595,2020-03-30
154595,35388218,2020-03-31


In [175]:
# добавляем столбец с дотой установки в основной дасафрейм 
data = data.merge(install_date, how = 'left', on='device_id')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01
...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06


In [176]:
# вычисляем дату первой покупки 
purch_date=data.loc[data.event=='purchase']\
               .groupby('device_id',as_index=False)\
               .date.min()\
               .rename(columns={'date':'purchase_date'})
purch_date

,device_id,purchase_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [177]:
# добавляем столбец с датой покупки в датафрейм

data = data.merge(purch_date, on='device_id', how='left')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT
...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31


In [178]:
# считаем кол-во дней прошедших со дня установки до покупки 

df1['days_to_purchase'] = (df1.purchase_date - df1.install_date).dt.days
df1

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,days_to_purchase,min_days
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0,6.0
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0,0.0
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN,NaN
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,NaN
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31,3.0,3.0
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31,0.0,0.0
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31,0.0,0.0
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31,25.0,25.0


In [179]:
# для каждого уникального девайса вычисляем дату первой покупки 

min_days = df1.groupby('device_id',as_index=False)\
              .days_to_purchase.min()\
              .rename(columns={'days_to_purchase':'min_days_to_purchase'})
min_days

,device_id,min_days_to_purchase
0,4013,NaN
1,4014,NaN
2,4016,NaN
3,4018,NaN
4,4046,0.0
...,...,...
190879,35379281,NaN
190880,35380796,NaN
190881,35381595,NaN
190882,35388218,NaN


In [180]:
# добавляем столбец с датой первой покупки в основной датафрейм

data = data.merge(min_days, on ='device_id', how='left')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31,3.0
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31,0.0
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31,0.0
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31,25.0


In [181]:
# так как у нас есть девайсы для коорых не указанна дата установки но есть покупки,оставим 
# только те данные где есть дата установки 

data1 = data.query('install_date!="NaT"')
data1

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN
5,2020-01-01,app_start,NaN,android,5148476,male,Saint-Petersburg,-,2020-01-01,2020-01-24,23.0
...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31,3.0
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31,0.0
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31,0.0
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31,25.0


In [182]:
# для дальнейшего вычисления конверсии в покупку нам понадобятся только 3 столбца 

cohorts_data = data1[['device_id','install_date','min_days_to_purchase']]
cohorts_data

,device_id,install_date,min_days_to_purchase
0,669460,2020-01-01,6.0
1,833621,2020-01-01,0.0
3,1737182,2020-01-01,NaN
4,4029024,2020-01-01,NaN
5,5148476,2020-01-01,23.0
...,...,...,...
2747963,2984778,2020-03-28,3.0
2747964,27301864,2020-03-31,0.0
2747965,1294285,2020-03-31,0.0
2747966,3010574,2020-03-06,25.0


In [183]:
# для каждого дня установки считаем количесво пользователей совершивших покупку в течение 7 дней 

purchases_in_7_days = cohorts_data.query('min_days_to_purchase <= 7')\
                                 .groupby('install_date',as_index=False)\
                                 .device_id.nunique()\
                                 .rename(columns={'device_id':'purchases_in_7_days'})
purchases_in_7_days

,install_date,purchases_in_7_days
0,2020-01-01,1408
1,2020-01-02,1186
2,2020-01-03,834
3,2020-01-04,639
4,2020-01-05,587
...,...,...
86,2020-03-27,277
87,2020-03-28,209
88,2020-03-29,223
89,2020-03-30,204


In [184]:
# для каждого дня установки считаем количесво пользователей устанвивших приложение в этот день 

installs_sum = cohorts_data.groupby('install_date',as_index=False)\
                           .device_id.nunique()\
                           .rename(columns={'device_id':'installs'})
installs_sum

,install_date,installs
0,2020-01-01,3579
1,2020-01-02,3144
2,2020-01-03,2402
3,2020-01-04,1831
4,2020-01-05,1671
...,...,...
86,2020-03-27,1199
87,2020-03-28,1091
88,2020-03-29,1117
89,2020-03-30,994


In [185]:
# объединяем данные о покупках за первые 7 дней после установки и суммарном кол-ве установок
cohorts_data_final = installs_sum.merge(purchases_in_7_days, on='install_date')
cohorts_data_final

,install_date,installs,purchases_in_7_days
0,2020-01-01,3579,1408
1,2020-01-02,3144,1186
2,2020-01-03,2402,834
3,2020-01-04,1831,639
4,2020-01-05,1671,587
...,...,...,...
86,2020-03-27,1199,277
87,2020-03-28,1091,209
88,2020-03-29,1117,223
89,2020-03-30,994,204


In [186]:
# считаем конверсию 
cohorts_data_final['conversion_rate'] = round(cohorts_data_final.purchases_in_7_days / cohorts_data_final.installs * 100, 2)
cohorts_data_final

,install_date,installs,purchases_in_7_days,conversion_rate
0,2020-01-01,3579,1408,39.34
1,2020-01-02,3144,1186,37.72
2,2020-01-03,2402,834,34.72
3,2020-01-04,1831,639,34.90
4,2020-01-05,1671,587,35.13
...,...,...,...,...
86,2020-03-27,1199,277,23.10
87,2020-03-28,1091,209,19.16
88,2020-03-29,1117,223,19.96
89,2020-03-30,994,204,20.52


In [187]:
cohorts_data_final.sort_values('conversion_rate',ascending=False).head(1)

,install_date,installs,purchases_in_7_days,conversion_rate
0,2020-01-01,3579,1408,39.34


Наибольшая конверсия в покупку за первые 7 дней после утановки у когорты пользователей установивших приложение 2020-01-01

____

## Задача 4

С какого платного маркетингового канала пришло больше всего новых пользователей?

Новым пользователем будем считать пользователей у которых в логах есть ивент установка

In [188]:
data.query('event=="app_install"').groupby('utm_source', as_index=False)\
                                  .device_id.nunique().sort_values('device_id', ascending=False)

,utm_source,device_id
0,-,32460
6,yandex-direct,29368
2,google_ads,26286
5,vk_ads,23189
3,instagram_ads,20096
1,facebook_ads,13916
4,referal,9282


Больше всего пользователей пришло из яндекс-директа - 29368

## Задача 5

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

In [189]:
# Дата регистрации пользователя
reg_date = data.loc[data.event=='register']\
               .groupby('device_id',as_index=False)\
               .date.min()\
               .rename(columns={'date':'reg_date'})

reg_date

,device_id,reg_date
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4067,2020-01-31
4,4120,2020-01-07
...,...,...
78305,35342310,2020-03-27
78306,35358658,2020-03-27
78307,35367797,2020-03-29
78308,35370030,2020-03-31


In [190]:
data = data.merge(reg_date, how='left', on='device_id')
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase,reg_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,NaT


In [192]:
# отбираем только зарегистрированных пользователей 
registrated_users = data.loc[data.reg_date <= data.date]
registrated_users

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase,reg_date
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,2020-01-01
8,2020-01-01,app_start,NaN,ios,10144701,female,Moscow,-,2020-01-01,2020-01-01,0.0,2020-01-01
10,2020-01-01,app_start,NaN,android,11876266,female,Moscow,yandex-direct,2020-01-01,NaT,NaN,2020-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31,3.0,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31,0.0,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31,0.0,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31,25.0,2020-03-31


In [194]:
registrated_users.groupby('event', as_index=False).device_id.nunique()

,event,device_id
0,app_install,42448
1,app_start,78289
2,choose_item,78310
3,purchase,70875
4,register,78310
5,search,78310
6,tap_basket,78310


## Задача 6
Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

In [195]:
# посчитаем количество установок по каналам
source_install = data[data.event == 'app_install'][['device_id', 'utm_source']]
source_install_num = source_install.groupby('utm_source', as_index=False) \
                                   .agg({'device_id': 'nunique'}) \
                                   .rename(columns={'device_id': 'install_num'})
source_install_num

,utm_source,install_num
0,-,32460
1,facebook_ads,13916
2,google_ads,26286
3,instagram_ads,20096
4,referal,9282
5,vk_ads,23189
6,yandex-direct,29368


In [196]:
# посчитаем пользователей совершивших хотя бы одну покупку по каналам
source_purchase = data[data.event == 'purchase'][['device_id', 'utm_source']].drop_duplicates()
source_purchase_num = source_purchase.groupby('utm_source', as_index=False) \
                                     .agg({'device_id': 'count'}) \
                                     .rename(columns={'device_id': 'purchase_num'})
source_purchase_num

,utm_source,purchase_num
0,-,16598
1,facebook_ads,9017
2,google_ads,11339
3,instagram_ads,10762
4,referal,6362
5,vk_ads,12364
6,yandex-direct,12028


In [197]:
# объединим данные
source_cr = source_install_num.merge(source_purchase_num, on='utm_source', how='inner')
source_cr

,utm_source,install_num,purchase_num
0,-,32460,16598
1,facebook_ads,13916,9017
2,google_ads,26286,11339
3,instagram_ads,20096,10762
4,referal,9282,6362
5,vk_ads,23189,12364
6,yandex-direct,29368,12028


In [198]:
# посчитаем конверсию
source_cr['cr'] = source_cr.purchase_num * 100 / source_cr.install_num
source_cr

,utm_source,install_num,purchase_num,cr
0,-,32460,16598,51.133703
1,facebook_ads,13916,9017,64.795918
2,google_ads,26286,11339,43.137031
3,instagram_ads,20096,10762,53.552946
4,referal,9282,6362,68.541263
5,vk_ads,23189,12364,53.318384
6,yandex-direct,29368,12028,40.956143


In [200]:
source_cr.loc[source_cr.cr.idxmin()].utm_source

'yandex-direct'

_____

## Задача 7

Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)



In [201]:
data.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase,reg_date
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,2020-01-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,NaT


In [211]:
first_purch = data.query('event=="purchase"')\
                  .groupby('device_id',as_index=False)\
                  .date.min().rename(columns={'date':'date_first_purch'})
first_purch

,device_id,date_first_purch
0,4014,2020-02-01
1,4046,2020-01-04
2,4060,2020-03-07
3,4120,2020-01-07
4,4125,2020-01-07
...,...,...
70870,35321120,2020-03-25
70871,35326839,2020-03-28
70872,35342310,2020-03-27
70873,35367797,2020-03-29


In [212]:
data = data.merge(first_purch, how='left', on='device_id')
data

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,install_date,purchase_date,min_days_to_purchase,reg_date,date_first_purch
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01-01,2020-01-07,6.0,2020-01-07,2020-01-07
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01-01,2020-01-01,0.0,2020-01-01,2020-01-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,NaT,2020-01-01,NaN,2020-01-01,2020-01-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,2020-01-01,NaT
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01-01,NaT,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads,2020-03-28,2020-03-31,3.0,2020-03-31,2020-03-31
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-,2020-03-31,2020-03-31,0.0,2020-03-31,2020-03-31
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads,2020-03-31,2020-03-31,0.0,2020-03-31,2020-03-31
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads,2020-03-06,2020-03-31,25.0,2020-03-31,2020-03-31


In [216]:
data.query('event=="purchase" & date==date_first_purch')\
    .groupby('utm_source',as_index=False)\
    .purchase_sum\
    .median()\
    .sort_values('purchase_sum')

,utm_source,purchase_sum
1,facebook_ads,389.0
2,google_ads,390.5
6,yandex-direct,392.5
5,vk_ads,393.0
3,instagram_ads,393.5
4,referal,395.5
0,-,398.5


Наибольший медианный чек первой покупки у пользователей пришедщших по реферальной программе - 395.5

____

## Задача 8

Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.


Какой платный канал привлечения имеет самый высокий ROMI? 

In [217]:
# посчитаем суммарный доход по каналам
source_revenue = data.groupby('utm_source', as_index=False) \
                     .agg({'purchase_sum': 'sum'})
source_revenue

,utm_source,purchase_sum
0,-,21449749.5
1,facebook_ads,12249901.0
2,google_ads,12868276.0
3,instagram_ads,14546969.0
4,referal,8837044.5
5,vk_ads,16389652.5
6,yandex-direct,13915368.0


In [218]:
# создадим из данных по затратам на рекламу датафрейм
source_cost = pd.DataFrame({'utm_source': ['facebook_ads', 'google_ads', 'instagram_ads', 'vk_ads', 'yandex-direct'],
                    'cost_sum': [8590498, 10534878, 8561626, 9553531, 10491707]})
source_cost

,utm_source,cost_sum
0,facebook_ads,8590498
1,google_ads,10534878
2,instagram_ads,8561626
3,vk_ads,9553531
4,yandex-direct,10491707


In [219]:
# объединим таблицы
romi_calc = source_revenue.merge(source_cost, on='utm_source', how='inner')
romi_calc

,utm_source,purchase_sum,cost_sum
0,facebook_ads,12249901.0,8590498
1,google_ads,12868276.0,10534878
2,instagram_ads,14546969.0,8561626
3,vk_ads,16389652.5,9553531
4,yandex-direct,13915368.0,10491707


In [220]:
# просчитаем ROMI
romi_calc['romi'] = ((romi_calc.purchase_sum - romi_calc.cost_sum) * 100 / romi_calc.cost_sum).round(1)
romi_calc

,utm_source,purchase_sum,cost_sum,romi
0,facebook_ads,12249901.0,8590498,42.6
1,google_ads,12868276.0,10534878,22.1
2,instagram_ads,14546969.0,8561626,69.9
3,vk_ads,16389652.5,9553531,71.6
4,yandex-direct,13915368.0,10491707,32.6


In [221]:
# Выберем канал с самым высоким ROMI
romi_calc.loc[romi_calc.romi.idxmax()].utm_source

'vk_ads'